## Face extraction using visual words

---

In this notebook I will use bag-of-visual generated by the [last notebook](https://colab.research.google.com/drive/1J5B1rTAGaAfFelf8P9d4lXzjjH1j_WBr?usp=sharing) and apply multiple models in order to select the best for bounding box. 

---

references:
  - sklearn for multiregression : [models](https://scikit-learn.org/stable/modules/multiclass.html)
  - save and load models : [post](https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/)

In [1]:
!gdown 10tken_6i_lw1lGZ0ZBiqQ8B47G44k-Nb

Downloading...
From: https://drive.google.com/uc?id=10tken_6i_lw1lGZ0ZBiqQ8B47G44k-Nb
To: /content/celeb_with_visual_words.csv
100% 16.5M/16.5M [00:00<00:00, 40.8MB/s]


In [2]:
!mkdir models

In [3]:
from sklearn.metrics import classification_report,plot_confusion_matrix,confusion_matrix
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model

from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC 
from sklearn import metrics
import plotly.express as px
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import pickle
import ast


tqdm.pandas()
sns.set_theme()
MODEL_DIR_NAME = 'models/'

In [4]:
def validation_metrics(y_pred, y_true, show_img=True):
  rmse = np.sqrt(np.square(y_pred - y_true).mean(axis=-1)) * 480
  fig = px.histogram(rmse)
  fig.show()
  return np.mean(rmse)

In [5]:
df = pd.read_csv('celeb_with_visual_words.csv')
df['visual_words_histogram'] = [np.array(ast.literal_eval(cur_vword)) for cur_vword in df['visual_words_histogram']]
df[['x0', 'y0', 'x1', 'y1']] = df[['x0', 'y0', 'x1', 'y1']] / 480 
df['visual_words_histogram'] = df['visual_words_histogram'].apply(lambda x: x/sum(x))
df.head(3)

,img_location,x0,y0,x1,y1,visual_words_histogram
0,celeb_data_resized/000001.jpg,0.232274,0.103348,0.784841,0.558952,"[0.005574912891986063, 0.006968641114982578, 0..."
1,celeb_data_resized/000002.jpg,0.170213,0.158249,0.692671,0.673401,"[0.0022753128555176336, 0.0022753128555176336,..."
2,celeb_data_resized/000003.jpg,0.432000,0.209964,0.614000,0.658363,"[0.00390625, 0.0, 0.0, 0.0, 0.00390625, 0.0, 0..."


In [6]:
df_train, df_test = train_test_split(df, test_size=0.3, random_state=42)

In [7]:
len(df_train), len(df_test)

(7028, 3012)

In [8]:
x_train = np.array(list(df_train['visual_words_histogram'].values))
y_train = np.array(list(df_train[['x0', 'y0', 'x1', 'y1']].values))

x_test = np.array(list(df_test['visual_words_histogram'].values))
y_test = np.array(list(df_test[['x0', 'y0', 'x1', 'y1']].values))

In [ ]:
def save_keras_model(model_name, model):
  model.save('models/' + model_name)

## Defining the models for bounding box generation

---

Now we define a model predict the bounding boxes of the image based on bag of visual words. 

So first we can try some Keras NN :

 - Simple dense nn
 - Simple dense nn with skip connections


In order to check the model performance we generate the histogram of RMSE of each example, them we display the mean of the RMSE distribution. The perfect model will have a mean RMSE equal to zero and a histogram concentrated around zero.

In [ ]:
from tensorflow.keras.layers import *
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Model
from keras.utils.vis_utils import plot_model


#### Simple dense NN

---

In [ ]:
def simple_dense_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])
  x = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x)
  x = Dropout(dropout)(x)
  outputs = Dense(num_classes, activation="sigmoid")(x)

  ret_model = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  ret_model.compile(opt, "mean_squared_error")
  return ret_model

In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
simple_dense = simple_dense_model(dropout=0.7,n_neurons=64, lr=1e-4)
simple_dense.fit(x_train,y_train, batch_size=64, epochs=500, 
                validation_split=0.2, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
88/88 [==============================] - 3s 5ms/step - loss: 0.0738 - val_loss: 0.0694
Epoch 2/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0631 - val_loss: 0.0557
Epoch 3/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0486 - val_loss: 0.0389
Epoch 4/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0358 - val_loss: 0.0264
Epoch 5/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0285 - val_loss: 0.0210
Epoch 6/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0260 - val_loss: 0.0194
Epoch 7/500
88/88 [==============================] - 0s 5ms/step - loss: 0.0252 - val_loss: 0.0189
Epoch 8/500
88/88 [==============================] - 1s 6ms/step - loss: 0.0247 - val_loss: 0.0187
Epoch 9/500
88/88 [==============================] - 0s 5ms/step - loss: 0.0243 - val_loss: 0.0186
Epoch 10/500
88/88 [==============================] - 0s 3ms/step - loss: 0.0241 - val_loss: 0.0185
Epoch 11/

In [ ]:
save_keras_model('simple_dense', simple_dense)

INFO:tensorflow:Assets written to: models/simple_dense/assets


In [ ]:
pred = simple_dense.predict(x_test)
validation_metrics(pred, y_test)

54.542866960712544

### Dense with skip connections

---

In [ ]:
def simple_dense_with_skip_model(dropout=0.5,n_neurons=64, num_classes=4, lr=1e-4):

  inputs = Input(shape=x_train[0].shape[0])

  x0 = Dense(n_neurons, activation="relu")(inputs)
  x = Dense(n_neurons, activation="relu")(x0)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dense(n_neurons, activation="relu")(x0)
  x = Dense(n_neurons, activation="relu")(x)
  x0 = Add()([x, x0])

  x = Dropout(dropout)(x0)

  outputs = Dense(num_classes, activation="sigmoid")(x)

  simple_dense_skip = Model(inputs=inputs, outputs=outputs)

  opt = Adam(learning_rate=lr)
  simple_dense_skip.compile(opt, "mean_squared_error")
  return simple_dense_skip

In [ ]:
dense_with_skip = simple_dense_with_skip_model(dropout=0.7,n_neurons=512, lr=1e-4)
dense_with_skip.fit(x_train,y_train, batch_size=128, epochs=500, 
                    validation_split=0.1, shuffle=True, verbose=True,  callbacks=[callback])

Epoch 1/500
50/50 [==============================] - 1s 7ms/step - loss: 0.0439 - val_loss: 0.0184
Epoch 2/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0207 - val_loss: 0.0182
Epoch 3/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0201 - val_loss: 0.0181
Epoch 4/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0197 - val_loss: 0.0179
Epoch 5/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0192 - val_loss: 0.0177
Epoch 6/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0192 - val_loss: 0.0175
Epoch 7/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0186 - val_loss: 0.0169
Epoch 8/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0175 - val_loss: 0.0162
Epoch 9/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0168 - val_loss: 0.0157
Epoch 10/500
50/50 [==============================] - 0s 4ms/step - loss: 0.0162 - val_loss: 0.0156
Epoch 11/

In [ ]:
save_keras_model('dense_with_skip', dense_with_skip)

INFO:tensorflow:Assets written to: models/dense_with_skip/assets


In [ ]:
pred = dense_with_skip.predict(x_test)
validation_metrics(pred, y_test)

53.58612386176372

In [ ]:
!zip -r models_NN_for_face_extraction.zip models

  adding: models/ (stored 0%)
  adding: models/dense_with_skip/ (stored 0%)
  adding: models/dense_with_skip/keras_metadata.pb (deflated 92%)
  adding: models/dense_with_skip/saved_model.pb (deflated 90%)
  adding: models/dense_with_skip/variables/ (stored 0%)
  adding: models/dense_with_skip/variables/variables.data-00000-of-00001 (deflated 11%)
  adding: models/dense_with_skip/variables/variables.index (deflated 71%)
  adding: models/dense_with_skip/assets/ (stored 0%)
  adding: models/simple_dense/ (stored 0%)
  adding: models/simple_dense/keras_metadata.pb (deflated 87%)
  adding: models/simple_dense/saved_model.pb (deflated 88%)
  adding: models/simple_dense/variables/ (stored 0%)
  adding: models/simple_dense/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: models/simple_dense/variables/variables.index (deflated 61%)
  adding: models/simple_dense/assets/ (stored 0%)
